In [1]:
import re
import requests
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import urllib
from urllib.parse import urlparse, urljoin
import colorama
import random
from stem.control import Controller
from stem import Signal
import sys

In [2]:
# print(sys.executable)
# print(sys.version)
# print(sys.version_info)

In [3]:
# init the colorama module
colorama.init()
GREEN = colorama.Fore.GREEN
GRAY = colorama.Fore.LIGHTBLACK_EX
RESET = colorama.Fore.RESET
YELLOW = colorama.Fore.YELLOW

In [8]:
def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)


def get_all_website_links(url, DynamicSite=0, verbose=False):
    """
    Returns all URLs that is found on `url` in which it belongs to the same website
    """
    
    urls = set()     # all URLs of `url`
    
    domain_name = urlparse(url).netloc     # domain name of the URL without the protocol

    if DynamicSite:
        session = HTMLSession()   # initialize an HTTP session
        
        response = session.get(url)  # make HTTP request & retrieve response
        
        # execute Javascript
        try:
            response.html.render()
        except:
            pass
        
        html_doc = response.html.html
        
    else :
        r = requests.get(url, allow_redirects=True)
        html_doc = r.content  # r.content    r.text
    

    
    soup = BeautifulSoup( html_doc , "html5lib")  # 'html5lib' , 'html.parser'  'lxml'
    
    
    #for link in soup.find_all(attrs={'href': re.compile("http")}):
    for link in soup.find_all('a', href=True):
        
        href = link.get('href')     #  link.attrs.get("href")   
        
        if href == "" or href is None:   
            continue   # href empty tag
            
        href = urljoin(url, href)      # join the URL if it's relative (not absolute link)
        
        parsed_href = urlparse(href)
        
        
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path  # remove URL GET parameters, URL fragments, etc.
        
        if not is_valid(href):     # not a valid URL
            continue      
            
        if href in internal_urls:  # already in the set
            continue       
            
        if domain_name not in href:  # external link
            if href not in external_urls:
                if verbose :
                    print(f"{GRAY}[!] External link: {href}{RESET}")
                external_urls.add(href)
            continue
            
        if verbose :
            print(f"{GREEN}[*] Internal link: {href}{RESET}")
        internal_urls.add(href)
        
        urls.add(href)
        
    return urls


def crawl(url, max_urls=30, DynamicSite=0, verbose=False):
    """
    Crawls a web page and extracts all links.
    You'll find all links in `external_urls` and `internal_urls` global set variables.
    params:
        max_urls (int): number of max urls to crawl, default is 30.
    """
    global total_urls_visited
    total_urls_visited += 1
    print(f"{YELLOW}[*] Crawling: {url}{RESET}")
    links = get_all_website_links(url, DynamicSite, verbose)
    for link in links:
        if total_urls_visited > max_urls:
            break
        crawl(link, max_urls, DynamicSite, verbose )

In [21]:
# url = "https://www.geeksforgeeks.org/"
# url = "https://www.varzesh3.com/"
url = "https://iut.ac.ir/"

In [10]:
# initialize the set of links (unique links)
internal_urls = set()
external_urls = set()

total_urls_visited = 0

crawl(url, 10)

[*] Crawling: https://iut.ac.ir/
[*] Crawling: http://news.iut.ac.ir/node/30645 
[*] Crawling: http://news.iut.ac.ir/contact-us
[*] Crawling: http://news.iut.ac.ir/send-news
[*] Crawling: http://news.iut.ac.ir/node/add
[*] Crawling: http://news.iut.ac.ir/archive-announcements
[*] Crawling: http://news.iut.ac.ir/content/%D8%AB%D8%A8%D8%AA%E2%80%8C%D9%86%D8%A7%D9%85-%D8%A2%D8%B2%D9%85%D9%88%D9%86-%D8%A7%D8%B3%D8%AA%D8%AE%D8%AF%D8%A7%D9%85%DB%8C-%D8%B3%D8%A7%D9%84-%DB%B1%DB%B4%DB%B0%DB%B0-%D8%AF%D8%A7%D9%86%D8%B4%DA%AF%D8%A7%D9%87%E2%80%8C%D9%87%D8%A7-%D9%88-%D9%85%D8%A4%D8%B3%D8%B3%D8%A7%D8%AA-%D9%88%D8%B2%D8%A7%D8%B1%D8%AA-%D8%B9%D9%84%D9%88%D9%85%D8%8C-%D8%AA%D8%AD%D9%82%DB%8C%D9%82%D8%A7%D8%AA-%D9%88-%D9%81%D9%86%D8%A2%D9%88%D8%B1%DB%8C
[*] Crawling: http://news.iut.ac.ir/content/%D8%A7%D8%B7%D9%84%D8%A7%D8%B9%DB%8C%D9%87-%D9%86%DA%AF%D8%B1%D8%B4-%D8%B3%D9%86%D8%AC%DB%8C-%D8%AF%D8%B1%D9%88%D8%B3-%D8%AF%D8%B1-%D9%86%DB%8C%D9%85%D8%B3%D8%A7%D9%84-%DB%B2-%DB%B9%DB%B9
[*] Crawling: http:/

In [ ]:
if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser(description="Link Extractor Tool with Python")
    parser.add_argument("url", help="The URL to extract links from.")
    parser.add_argument("-m", "--max-urls", help="Number of max URLs to crawl, default is 30.", default=30, type=int)
    
    args = parser.parse_args()
    url = args.url
    max_urls = args.max_urls

    crawl(url, max_urls=max_urls)

    print("[+] Total Internal links:", len(internal_urls))
    print("[+] Total External links:", len(external_urls))
    print("[+] Total URLs:", len(external_urls) + len(internal_urls))
    print("[+] Total crawled URLs:", max_urls)

    domain_name = urlparse(url).netloc

    # save the internal links to a file
    with open(f"{domain_name}_internal_links.txt", "w") as f:
        for internal_link in internal_urls:
            print(internal_link.strip(), file=f)

    # save the external links to a file
    with open(f"{domain_name}_external_links.txt", "w") as f:
        for external_link in external_urls:
            print(external_link.strip(), file=f)

In [ ]:
import sublist3r 

In [ ]:
# engines='baidu,yahoo,google,bing,ask,netcraft,dnsdumpster,virustotal,threatcrowd,ssl,passivedns'

subdomains = sublist3r.main(domain='varzesh3.com', 
                            threads=40, 
                            savefile='yahoo_subdomains.txt', 
                            ports= None, 
                            silent=False, 
                            verbose= False, 
                            enable_bruteforce= False, 
                            engines=None
                           )

## Using Tor as a Proxy

In [ ]:
def get_tor_session():
    session = requests.session()
    # Tor uses the 9050 port as the default socks port
    session.proxies = {'http':  'socks5://127.0.0.1:9050',
                       'https': 'socks5://127.0.0.1:9050'}
    return session

def renew_connection():
    with Controller.from_port(port=9051) as c:
        c.authenticate()
        # send NEWNYM signal to establish a new clean connection through the Tor network
        c.signal(Signal.NEWNYM)

In [ ]:
import requests
torport = 9050
proxies = {
    'http': "socks5h://localhost:{}".format(torport),
    'https': "socks5h://localhost:{}".format(torport)
}

print(requests.get('http://icanhazip.com', proxies=proxies).content)

In [ ]:
# Make a request through the Tor connection
# IP visible through Tor
session = get_tor_session()
print(session.get("http://httpbin.org/ip").text)
# Above should print an IP different than your public IP

# Following prints your normal public IP
print(requests.get("http://httpbin.org/ip").text)

In [ ]:
if __name__ == "__main__":
    s = get_tor_session()
    ip = s.get("http://icanhazip.com").text
    print("IP:", ip)
    renew_connection()
    s = get_tor_session()
    ip = s.get("http://icanhazip.com").text
    print("IP:", ip)

In [ ]:
# https://github.com/clarketm/proxy-list
def get_free_proxies():
    proxyURL = "https://raw.githubusercontent.com/clarketm/proxy-list/master/proxy-list-raw.txt"

    data = urllib.request.urlopen(proxyURL).read( 20*400 ).split()

    proxies = [ line.decode("utf-8") for line in data ]

    return proxies


def get_free_proxies2():
    url = "https://free-proxy-list.net/"
    # get the HTTP response and construct soup object
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    proxies = []
    for row in soup.find("table", attrs={"id": "proxylisttable"}).find_all("tr")[1:]:
        tds = row.find_all("td")
        try:
            ip = tds[0].text.strip()
            port = tds[1].text.strip()
            host = f"{ip}:{port}"
            proxies.append(host)
        except IndexError:
            continue
    return proxies

def get_session(proxies):
    # construct an HTTP session
    session = requests.Session()
    # choose one random proxy
    proxy = random.choice(proxies)
    session.proxies = {"http": 'http://'+proxy, "https": 'https://'+proxy}
    return session

In [ ]:
proxies = get_free_proxies()

for i in range(100):
    s = get_session(proxies)
    try:
        print("Request page with IP:", s.get("http://icanhazip.com", timeout=1.5).text.strip())
    except Exception as e:
        continue

In [ ]:
url = "https://www.geeksforgeeks.org"
# url = "https://www.varzesh3.com/"
url = "https://iut.ac.ir/"

In [ ]:
parsed = urlparse(url)
parsed

In [ ]:
urljoin("https://iut.ac.ir/ab/", "http://indust.iut.ac.ir")

In [ ]:
print( len( soup.find_all('a', href=True) ) )
print( len( soup.find_all(attrs={'href': re.compile("http")}) ) )

In [ ]:
if __name__ == '__main__':
    main()